In [1]:
import sys
sys.path.append('')
import json
import requests
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from pprint import pprint
from config import weather
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
import statsmodels.api as sm
from scipy import stats

In [2]:
data0 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag0')
data1 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag1')
data2 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag2')
data3 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag3')
data4 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag4')
data5 = pd.read_excel('Tornado_Data_2009to2015_byMag.xlsx','Mag5')
data0_sample = data0.sample(225)
data1_sample = data1.sample(127)
data2_sample = data2.sample(35)
data3_sample = data3.sample(11)
data4_sample = data4
data5_sample = data5

data = pd.concat([data0_sample, data1_sample, data2_sample, data3_sample, data4_sample, data5_sample])


In [3]:

test = data
test.shape
#test = data.iloc[25:51]

(459, 22)

In [4]:

#putting coordinates into a list
coordinates = test[['Starting Latitude', 'Starting Longitude']]
#print(coordinates)
coord_list = []
for index, row in coordinates.iterrows():
    y = row['Starting Latitude']
    z = row['Starting Longitude']
    a = (y,z)
    coord_list.append(a)


In [5]:
#putting dates into list

start_date = []
end_date = []

for index, row in test[['Year','Month','Day','Date']].iterrows():
    year = row['Year']
    month = row['Month']
    day = row['Day']
    e = datetime(year,month,day)-timedelta(3)
    e= e.strftime('%Y-%m-%d')
    end_date.append(e)
    s = datetime(year,month,day)
    s= s.strftime('%Y-%m-%d')
    start_date.append(s)

In [6]:
# df to add weather data to
f = pd.DataFrame()
f['Magnitude'] = test['Magnitude']
f['Start Date'] = start_date
f['End Date'] = end_date
f['coord'] = coord_list
f['Max Temp (F)'] = ''
f['Min Temp (F)'] = ''
f['Wind Speed (mph)'] = ''
#f['Precip (mm)'] = ''
f['Humidity'] = ''
f['Visibility'] = ''
f['Pressure'] = ''
f['Cloud Cover'] = ''
f['Heat Index (F)'] = ''
f['Dew Point (F)'] = ''
f['Wind Chill (F)'] = ''
f['Wind Gust (mph)'] = ''

In [7]:
for index,row in f.iterrows():
    coord = row['coord']
    start = row['Start Date']
    end = row['End Date']
    tp = '24'

    maxtF = []
    mintF = []
    wind = []
   # precip_list = []
    hum_list = []
    vis_list = []
    pres_list = []
    clou_list = []
    heat_list = []
    dew_list = []
    wind_chill_list = []
    wind_gust_list = []

    url = ('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={0}&q={1},{2}&format=json&date={3}&tp={4}'
          ).format(weather,coord[0],coord[1],start,tp)
    weather_data = requests.get(url).json()
    
    for i in range(0,1):
        max_temp = int(weather_data['data']['weather'][i]['maxtempF'])
        maxtF.append(max_temp)
        min_temp = int(weather_data['data']['weather'][i]['mintempF'])
        mintF.append(min_temp)
        wind_speed = int(weather_data['data']['weather'][i]['hourly'][0]['windspeedMiles'])
        wind.append(wind_speed)
        #precip = weather_data['data']['weather'][i]['hourly'][0]['precipMM']
        #precip_list.append(precip)
        humidity = int(weather_data['data']['weather'][i]['hourly'][0]['humidity'])
        hum_list.append(humidity)
        visibility = int(weather_data['data']['weather'][i]['hourly'][0]['visibility'])
        vis_list.append(visibility)
        pressure = int(weather_data['data']['weather'][i]['hourly'][0]['pressure'])
        pres_list.append(pressure)
        cloud_cover = int(weather_data['data']['weather'][i]['hourly'][0]['cloudcover'])
        clou_list.append(cloud_cover)
        heat_index = int(weather_data['data']['weather'][i]['hourly'][0]['HeatIndexF'])
        heat_list.append(heat_index)
        dew_point = int(weather_data['data']['weather'][i]['hourly'][0]['DewPointF'])
        dew_list.append(dew_point)
        wind_chill = int(weather_data['data']['weather'][i]['hourly'][0]['WindChillF'])
        wind_chill_list.append(wind_chill)
        wind_gust = int(weather_data['data']['weather'][i]['hourly'][0]['WindGustMiles'])
        wind_gust_list.append(wind_gust)
    max_temp_mean = pd.DataFrame({'max':maxtF}).mean()
    min_temp_mean = pd.DataFrame({'min':mintF}).mean()
    wind_speed_mean = pd.DataFrame({'wind_speed':wind}).mean()
    #precip_mean = pd.DataFrame({'precip':precip_list}).mean()
    hum_mean = pd.DataFrame({'hum':hum_list}).mean()
    visib_mean = pd.DataFrame({'vis':vis_list}).mean()
    pres_mean = pd.DataFrame({'pres': pres_list}).mean()
    clou_mean = pd.DataFrame({'cloud':clou_list}).mean()
    heat_index_mean = pd.DataFrame({'heat':heat_list}).mean()
    dew_mean = pd.DataFrame({'dew':dew_list}).mean()
    wind_chill_mean = pd.DataFrame({'wchill':wind_chill_list}).mean()
    wind_gust_mean = pd.DataFrame({'wgust':wind_gust_list}).mean()
    f.loc[index,'Max Temp (F)'] = int(max_temp_mean)
    f.loc[index, 'Min Temp (F)'] = int(min_temp_mean)
    f.loc[index, 'Wind Speed (mph)'] = int(wind_speed_mean)
    #f.loc[index, 'Precip (mm)'] = precip_mean
    f.loc[index, 'Humidity'] = int(hum_mean)
    f.loc[index, 'Visibility'] = int(visib_mean)
    f.loc[index, 'Pressure'] = int(pres_mean)
    f.loc[index, 'Cloud Cover'] = int(clou_mean)
    f.loc[index, 'Heat Index (F)'] = int(heat_index_mean)
    f.loc[index, 'Dew Point (F)'] = int(dew_mean)
    f.loc[index, 'Wind Chill (F)'] = int(wind_chill_mean)
    f.loc[index, 'Wind Gust (mph)'] = int(wind_gust_mean)
    

In [8]:
f.columns

Index(['Magnitude', 'Start Date', 'End Date', 'coord', 'Max Temp (F)',
       'Min Temp (F)', 'Wind Speed (mph)', 'Humidity', 'Visibility',
       'Pressure', 'Cloud Cover', 'Heat Index (F)', 'Dew Point (F)',
       'Wind Chill (F)', 'Wind Gust (mph)'],
      dtype='object')

In [9]:
y = f['Magnitude']
x = f[['Pressure', 'Dew Point (F)',
       'Wind Gust (mph)']]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = .2)

In [11]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

In [12]:
X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:              Magnitude   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     25.57
Date:                Tue, 22 Jan 2019   Prob (F-statistic):           2.64e-15
Time:                        17:05:57   Log-Likelihood:                -770.30
No. Observations:                 459   AIC:                             1549.
Df Residuals:                     455   BIC:                             1565.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              39.0149     12.919     

In [13]:
correlation = f.corr()
correlation

,Magnitude,Max Temp (F),Min Temp (F),Wind Speed (mph),Humidity,Visibility,Pressure,Cloud Cover,Heat Index (F),Dew Point (F),Wind Chill (F),Wind Gust (mph)
Magnitude,1.000000,0.074325,0.089962,0.339079,0.120951,-0.015878,-0.239391,0.135287,0.053409,0.136301,0.046325,0.327997
Max Temp (F),0.074325,1.000000,0.765338,-0.065026,-0.442825,0.325665,-0.111545,-0.446815,0.896344,0.591213,0.909371,-0.083115
Min Temp (F),0.089962,0.765338,1.000000,0.059647,-0.021568,0.076257,-0.037131,-0.080383,0.893565,0.843145,0.882900,0.092508
Wind Speed (mph),0.339079,-0.065026,0.059647,1.000000,-0.003942,0.099363,-0.408696,0.233876,-0.088618,-0.054168,-0.104697,0.928047
Humidity,0.120951,-0.442825,-0.021568,-0.003942,1.000000,-0.550248,0.121935,0.706455,-0.241897,0.362073,-0.283574,0.096757
Visibility,-0.015878,0.325665,0.076257,0.099363,-0.550248,1.000000,-0.054586,-0.519312,0.222715,-0.092816,0.226141,0.028239
Pressure,-0.239391,-0.111545,-0.037131,-0.408696,0.121935,-0.054586,1.000000,0.001784,-0.006352,0.058386,-0.025334,-0.369064
Cloud Cover,0.135287,-0.446815,-0.080383,0.233876,0.706455,-0.519312,0.001784,1.000000,-0.342890,0.075287,-0.352423,0.295384
Heat Index (F),0.053409,0.896344,0.893565,-0.088618,-0.241897,0.222715,-0.006352,-0.342890,1.000000,0.797898,0.980615,-0.078079
Dew Point (F),0.136301,0.591213,0.843145,-0.054168,0.362073,-0.092816,0.058386,0.075287,0.797898,1.000000,0.769628,0.005163
